## Requirement

In [1]:
#!pip install tqdm
#!pip install ydata-profiling
!pip install lightgbm
!pip install catboost
!pip install xgboost

## Import Library

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#from tqdm.notebook import tqdm
#from ydata_profiling import ProfileReport

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


## Import Dataset

In [3]:
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")
sub = pd.read_csv("/content/sample_submission.csv")

In [4]:
train

,Unnamed: 0,Negara/Tahun,Emisi Savanna Api,Emisi Kebakaran Hutan,Emisi Residu Tanaman,Emisi Budidaya Padi,Emisi Tanah Organik Yang Dikeringkan (Co2),Emisi Pembuatan Pestisida,Emisi Transportasi Makanan,Lahan Hutan,Konversi Hutan Bersih,Emisi Konsumsi Rumah Tangga Makanan,Emisi Ritel Makanan,Emisi Penggunaan Listrik Di Pertanian,Emisi Kemasan Makanan,Emisi Sistem Agrifood Pembuangan Limbah,Emisi Pengolahan Makanan,Emisi Manufaktur Pupuk,Emisi IPPU,Emisi Kotoran Diterapkan Pada Tanah,Emisi Pupuk Kandang Di Padang Rumput,Emisi Manajemen Pupuk,Emisi Kebakaran Di Tanah Organik,Emisi Kebakaran Di Hutan Tropis Yang Lembab,Penggunaan Energi Di Pertanian,Populasi Pedesaan,Populasi Perkotaan,Total Populasi - Pria,Total Populasi - Wanita,Emisi Total,Peningkatan Suhu Rata - Rata ° C
0,0,Afghanistan/1990,14.7237,0.0557,205.6077,686.0000,0.0,11.807483,63.1152,-2388.8030,0.0000,79.0851,109.6446,14.2666,67.631366,691.7888,252.21419,11.997000,209.9778,260.1431,1590.5319,319.1763,0.0,0.0,NaN,9655167.0,2593947.0,5348387.0,5346409.0,2198.963539,0.536167
1,1,Afghanistan/1991,14.7237,0.0557,209.4971,678.1600,0.0,11.712073,61.2125,-2388.8030,0.0000,80.4885,116.6789,11.4182,67.631366,710.8212,252.21419,12.853900,217.0388,268.6292,1657.2364,342.3079,0.0,0.0,NaN,10230490.0,2763167.0,5372959.0,5372208.0,2323.876629,0.020667
2,2,Afghanistan/1992,14.7237,0.0557,196.5341,686.0000,0.0,11.712073,53.3170,-2388.8030,0.0000,80.7692,126.1721,9.2752,67.631366,743.6751,252.21419,13.492900,222.1156,264.7898,1653.5068,349.1224,0.0,0.0,NaN,10995568.0,2985663.0,6028494.0,6028939.0,2356.304229,-0.259583
3,3,Afghanistan/1993,14.7237,0.0557,230.8175,686.0000,0.0,11.712073,54.3617,-2388.8030,0.0000,85.0678,81.4607,9.0635,67.631366,791.9246,252.21419,14.055900,201.2057,261.7221,1642.9623,352.2947,0.0,0.0,NaN,11858090.0,3237009.0,7003641.0,7000119.0,2368.470529,0.101917
4,4,Afghanistan/1994,14.7237,0.0557,242.0494,705.6000,0.0,11.712073,53.9874,-2388.8030,0.0000,88.8058,90.4008,8.3962,67.631366,831.9181,252.21419,15.126900,182.2905,267.6219,1689.3593,367.6784,0.0,0.0,NaN,12690115.0,3482604.0,7733458.0,7722096.0,2500.768729,0.372250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5598,5598,Zimbabwe/2010,2795.6192,283.6316,109.3055,3.0243,0.0,87.000000,157.6366,262.6108,10677.6439,141.1969,340.6886,260.8153,28.342500,1004.2754,64.36670,1227.240253,709.7960,103.6080,2968.0562,302.3508,0.0,0.0,2074.1869,9410211.0,4676106.0,6034165.0,6805605.0,23601.395453,0.911917
5599,5599,Zimbabwe/2011,2918.2098,168.1659,100.7798,3.7456,0.0,94.000000,268.6740,648.0808,10670.8870,179.8443,328.7726,331.5380,26.920300,1019.7733,286.50940,1127.687805,843.8980,102.2926,3072.2327,312.2478,0.0,0.0,966.3719,9636932.0,4749717.0,6114111.0,6911674.0,23470.631605,0.191167
5600,5600,Zimbabwe/2012,2164.8953,259.4249,103.8422,5.5527,0.0,91.000000,304.6578,648.0808,10670.8870,170.8670,322.0831,317.3356,25.000800,1037.0116,280.77900,1127.687805,947.9330,102.4605,3101.1270,314.3433,0.0,0.0,908.2629,9880721.0,4830105.0,6223803.0,7041528.0,22903.232305,0.337000
5601,5601,Zimbabwe/2013,1544.9329,238.1898,96.8978,5.8016,0.0,73.000000,338.5506,648.0808,10670.8870,203.6878,335.2805,625.2430,22.669900,1045.1572,304.91080,1127.687805,869.9150,103.5893,3127.0658,317.9359,0.0,0.0,546.0138,10138667.0,4915839.0,6363142.0,7192279.0,22245.497305,0.089667


In [5]:
test

,Unnamed: 0,Negara/Tahun,Emisi Savanna Api,Emisi Kebakaran Hutan,Emisi Residu Tanaman,Emisi Budidaya Padi,Emisi Tanah Organik Yang Dikeringkan (Co2),Emisi Pembuatan Pestisida,Emisi Transportasi Makanan,Lahan Hutan,Konversi Hutan Bersih,Emisi Konsumsi Rumah Tangga Makanan,Emisi Ritel Makanan,Emisi Penggunaan Listrik Di Pertanian,Emisi Kemasan Makanan,Emisi Sistem Agrifood Pembuangan Limbah,Emisi Pengolahan Makanan,Emisi Manufaktur Pupuk,Emisi IPPU,Emisi Kotoran Diterapkan Pada Tanah,Emisi Pupuk Kandang Di Padang Rumput,Emisi Manajemen Pupuk,Emisi Kebakaran Di Tanah Organik,Emisi Kebakaran Di Hutan Tropis Yang Lembab,Penggunaan Energi Di Pertanian,Populasi Pedesaan,Populasi Perkotaan,Total Populasi - Pria,Total Populasi - Wanita,Emisi Total
0,0,Afghanistan/2015,0.8454,0.0000,403.3181,642.8800,0.0,81.852555,440.0315,-246.2191,0.0000,1570.3394,370.6039,75.1614,353.93064,3090.5068,1645.129815,27.235400,256.7030,423.5038,2719.1528,797.9007,0.0,0.0,NaN,25368923.0,8367571.0,17071446.0,16682054.0,12652.876111
1,1,Afghanistan/2016,1.6558,0.0000,387.6130,466.4800,0.0,54.909681,340.8931,154.6574,0.0000,1649.8382,425.9346,84.2970,353.93064,3201.0961,1627.654458,27.846900,302.7560,421.9268,2692.9570,793.9377,0.0,0.0,NaN,25985093.0,8670939.0,17520861.0,17115346.0,12988.384380
2,2,Afghanistan/2017,0.4015,0.0000,344.6447,429.0518,0.0,55.148427,345.7609,154.6574,0.0000,1431.5300,477.3314,96.5916,353.93064,3306.1192,1609.985794,35.309400,273.8355,422.4022,2680.8381,768.6802,0.0,0.0,NaN,26558609.0,8971472.0,18028696.0,17614722.0,12786.218762
3,3,Afghanistan/2018,0.2008,0.0000,291.7838,460.7529,0.0,72.743150,407.6310,154.6574,0.0000,1392.5480,534.8263,83.0929,353.93064,3406.9936,1627.654458,35.501200,298.9150,427.5072,2716.9079,789.3364,0.0,0.0,NaN,27099874.0,9273302.0,18549862.0,18136922.0,13054.982649
4,4,Afghanistan/2019,7.1050,0.0000,395.2689,499.9176,0.0,80.806938,489.7252,154.6574,0.0000,1520.2581,590.6114,92.4828,353.93064,3503.2037,1609.985794,35.554900,308.9680,387.9500,2557.4329,766.5012,0.0,0.0,NaN,27626382.0,9582625.0,19090409.0,18679089.0,13354.360473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1357,1357,Zimbabwe/2016,1190.0089,232.5068,70.9451,7.4088,0.0,75.000000,251.1465,76500.2982,10662.4408,251.2681,443.0872,428.4352,22.91080,1077.2392,317.074400,2585.080847,858.9820,96.1332,2721.1459,282.5994,0.0,0.0,417.3150,10934468.0,5215894.0,6796658.0,7656047.0,98491.026347
1358,1358,Zimbabwe/2017,1431.1407,131.1324,108.6262,7.9458,0.0,67.000000,255.7975,76500.2982,10662.4408,203.1236,445.3881,304.7852,18.98570,1093.3441,332.775900,1227.240253,889.4250,81.2314,2744.8763,255.5900,0.0,0.0,398.1644,11201138.0,5328766.0,6940631.0,7810471.0,97159.311553
1359,1359,Zimbabwe/2018,1557.5830,221.6222,109.9835,8.1399,0.0,66.000000,327.0897,76500.2982,10662.4408,211.1539,492.8599,346.8512,19.05700,1108.5523,348.510700,1127.687805,966.2650,81.0712,2790.0949,257.2735,0.0,0.0,465.7735,11465748.0,5447513.0,7086002.0,7966181.0,97668.308205
1360,1360,Zimbabwe/2019,1591.6049,171.0262,45.4574,7.8322,0.0,73.000000,290.1893,76500.2982,10662.4408,228.6381,542.5922,350.2168,17.95140,1121.3255,327.820900,2485.528399,945.9420,85.7211,2828.7215,267.5224,0.0,0.0,444.2335,11725970.0,5571525.0,7231989.0,8122618.0,98988.062799


In [6]:
sub

,id,Peningkatan Suhu Rata - Rata ° C
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
1357,1357,0
1358,1358,0
1359,1359,0
1360,1360,0


## EDA (1)

In [7]:
#profile = ProfileReport(train,title = 'Temperature Prediction')
#profile.to_notebook_iframe()
#profile.to_file("temperature_pred.html")

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5603 entries, 0 to 5602
Data columns (total 31 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Unnamed: 0                                   5603 non-null   int64  
 1   Negara/Tahun                                 5603 non-null   object 
 2   Emisi Savanna Api                            5578 non-null   float64
 3   Emisi Kebakaran Hutan                        5528 non-null   float64
 4   Emisi Residu Tanaman                         4485 non-null   float64
 5   Emisi Budidaya Padi                          5603 non-null   float64
 6   Emisi Tanah Organik Yang Dikeringkan (Co2)   5603 non-null   float64
 7   Emisi Pembuatan Pestisida                    5603 non-null   float64
 8   Emisi Transportasi Makanan                   5603 non-null   float64
 9   Lahan Hutan                                  5206 non-null   float64
 10  

In [9]:
train.isnull().sum()

,0
Unnamed: 0,0
Negara/Tahun,0
Emisi Savanna Api,25
Emisi Kebakaran Hutan,75
Emisi Residu Tanaman,1118
Emisi Budidaya Padi,0
Emisi Tanah Organik Yang Dikeringkan (Co2),0
Emisi Pembuatan Pestisida,0
Emisi Transportasi Makanan,0
Lahan Hutan,397


In [10]:
df = train.isnull().sum()/len(train) * 100
df

,0
Unnamed: 0,0.000000
Negara/Tahun,0.000000
Emisi Savanna Api,0.446190
Emisi Kebakaran Hutan,1.338569
Emisi Residu Tanaman,19.953596
Emisi Budidaya Padi,0.000000
Emisi Tanah Organik Yang Dikeringkan (Co2),0.000000
Emisi Pembuatan Pestisida,0.000000
Emisi Transportasi Makanan,0.000000
Lahan Hutan,7.085490


In [11]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,5603.0,2.801000e+03,1.617591e+03,0.000000,1400.500000,2.801000e+03,4.201500e+03,5.602000e+03
Emisi Savanna Api,5578.0,1.229654e+03,5.472354e+03,0.000000,0.000000,1.657900e+00,1.143092e+02,1.146164e+05
Emisi Kebakaran Hutan,5528.0,9.242752e+02,3.692585e+03,0.000000,0.000000,6.618500e-01,7.109490e+01,5.222763e+04
Emisi Residu Tanaman,4485.0,9.492051e+02,3.498477e+03,0.000200,10.921100,9.803740e+01,3.562161e+02,3.063853e+04
Emisi Budidaya Padi,5603.0,4.241102e+03,1.759963e+04,0.000000,172.219300,5.155172e+02,1.551411e+03,1.649153e+05
Emisi Tanah Organik Yang Dikeringkan (Co2),5603.0,3.471792e+03,1.548866e+04,0.000000,0.000000,0.000000e+00,6.941645e+02,2.321185e+05
Emisi Pembuatan Pestisida,5603.0,3.275223e+02,1.445485e+03,0.000000,5.000000,1.232828e+01,1.069817e+02,1.645900e+04
Emisi Transportasi Makanan,5603.0,1.823138e+03,5.306418e+03,0.000100,26.262663,1.760653e+02,1.135650e+03,6.204817e+04
Lahan Hutan,5206.0,-1.814565e+04,8.104574e+04,-797183.079000,-3224.371500,-6.953460e+01,0.000000e+00,1.711211e+05
Konversi Hutan Bersih,5206.0,1.845452e+04,1.080050e+05,0.000000,0.000000,4.461600e+01,4.734968e+03,1.605106e+06


## Cleaning

In [12]:
train = train.drop(columns=["Unnamed: 0"])
test = test.drop(columns=["Unnamed: 0"])

In [13]:
train[["Negara", "Tahun"]] = train["Negara/Tahun"].str.split('/', expand=True)
train = train.drop(columns=['Negara/Tahun'])

test[["Negara", "Tahun"]] = test["Negara/Tahun"].str.split('/', expand=True)
test = test.drop(columns=['Negara/Tahun'])

In [14]:
encoder = LabelEncoder()
train[['Negara', 'Tahun']] = train[['Negara', 'Tahun']].apply(encoder.fit_transform)
test[['Negara', 'Tahun']] = test[['Negara', 'Tahun']].apply(encoder.fit_transform)

In [16]:
# 1. Fill using -1
train = train.fillna(-1)
test = test.fillna(-1)

# 2. Imputation using MICE
#mice = IterativeImputer(max_iter=10000, random_state=42)
#train = pd.DataFrame(mice.fit_transform(train), columns=train.columns)
#test = pd.DataFrame(mice.fit_transform(test), columns=test.columns)

# 3. Drop Missing Value Column
#train = train.dropna(axis=1)
#test = train.dropna(axis=1)

## EDA (2)

## Modelling

In [17]:
X = train.drop('Peningkatan Suhu Rata - Rata ° C', axis=1)
y = train['Peningkatan Suhu Rata - Rata ° C']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
models = {
    'XGBoost': XGBRegressor(random_state=42),
    'CatBoost': CatBoostRegressor(verbose=False, random_state=42),
    'LightGBM': LGBMRegressor(random_state=42)
}

In [20]:
results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mape = mean_absolute_percentage_error(y_test, y_pred)

    results[name] = mape
    print(f"{name} MAPE: {mape:.4f}")

XGBoost MAPE: 0.9347
CatBoost MAPE: 0.9254
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001826 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7150
[LightGBM] [Info] Number of data points in the train set: 4482, number of used features: 30
[LightGBM] [Info] Start training from score 0.751719
LightGBM MAPE: 0.9578


In [23]:
cb = CatBoostRegressor(verbose=False, random_state=42)
cb.fit(X_train, y_train)

pred = cb.predict(test)

In [24]:
sub['Peningkatan Suhu Rata - Rata ° C'] = pred
sub.to_csv('testing.csv', index=False)